# Benchmarking Cardinal Distance Algorithms

In [18]:
from typing import Tuple
import numpy as np
import random
from pympler import asizeof
from humanize import naturalsize

First we are going to look at my approach

My function does a shift-modulo-shift approach to handle board wrap. Other implementations seem to prefer the if-statement approach, but I'm always suspicious of if statements in highly optimized code because of branch prediction penalties. Modern processors do a lot of speculative execution, so it can often execute code more quickly if it doesn't need to branch execution depending on the result of a computation. At least that's true in C code. I honestly don't know how true that is in cpython when the code is interpreted by the VM.

In [2]:
def cardinal_distance_mine(start_pos: Tuple[int, int], end_pos:Tuple[int, int], boardsize=21):
    center = boardsize // 2
    dx = (end_pos[0] - start_pos[0] + center) % boardsize - center
    dy = (end_pos[1] - start_pos[1] + center) % boardsize - center
    return dx, dy

Next we are going to look at the approach I saw with some of the starter bots and I saw on stack overflow which is to use if statements. This one has the benefit of very fewer mathematical operations than mine on average but includes some conditional branches of execution.

In [3]:
def cardinal_distance_if(start_pos: Tuple[int, int], end_pos:Tuple[int, int], boardsize=21):
    center = boardsize // 2
    dx = end_pos[0] - start_pos[0]
    dy = end_pos[1] - start_pos[1]
    if dx > center:
        dx -= boardsize
    if dx < -center:
        dx += boardsize
    if dy > center:
        dy -= boardsize
    if dy < -center:
        dy += boardsize
    return dx, dy 

Bob's cardinal distance function is a bit more convoluted but it doesn't matter how complex it is because he only uses it to prefill an array that we index into.

In [4]:
def cardinal_distance(start_point, end_point, boardsize=21):
    # returns the distance needed to travel across a wrapped board of size [boardsize] where the 
    # first output is the west to east distance (or a negative value if faster to travel westbound)
    # and the second output is the north to south distance (or a negative value if shorter to 
    # travel southbound.
    #
    # The inputs, start_point and end_point are expected to be integers where value zero is the northwest
    # point on the board and value boardsize*boardsize-1 is the southeast point on the board.
    
    # Calculate the distance traveling east (1st element) or west (2nd element)
    dist_west2east = ((end_point - start_point) % boardsize, 
                      (boardsize - ( (end_point - start_point) % boardsize) ))
    # return the signed minimum distance, negative values means travel west
    dist_west2east = min(dist_west2east)*(-1)**dist_west2east.index(min(dist_west2east))

    # Calculate the distance traveling south (1st element) or north (2nd element)
    dist_north2south = ((end_point//boardsize - start_point//boardsize) % boardsize, 
                        ( boardsize - ( (end_point//boardsize - start_point//boardsize) % boardsize) ))
    # return the signed minimum distance, where negative values mean traveling north
    dist_north2south = min(dist_north2south)*(-1)**dist_north2south.index(min(dist_north2south))

    return dist_west2east, dist_north2south

def make_cardinal_distance_list(boardsize=21):
    startpoints = np.arange(boardsize**2)
    endpoints = np.arange(boardsize**2)
    cardinal_distance_list = []
    for start_point in startpoints:
        cardinal_distance_list.append([cardinal_distance(start_point,end_point) for end_point in endpoints])
    return cardinal_distance_list

cardinal_distance_list = make_cardinal_distance_list()


# Correctness
Let's compare the implementations to make sure they return the same results

In [5]:
test_cases = [(random.randint(0,21),random.randint(0, 21)) for _ in range(6)]
for p1, p2 in zip(test_cases, test_cases[1:]):
    print(p1, p2, '=>',
          cardinal_distance_mine(p1, p2),
          cardinal_distance_if(p1, p2),
          cardinal_distance(p1[0] + p1[1] * 21, p2[0] + p2[1] * 21))

(21, 7) (5, 6) => (5, -1) (5, -1) (5, -2)
(5, 6) (12, 13) => (7, 7) (7, 7) (7, 7)
(12, 13) (1, 15) => (10, 2) (10, 2) (10, 2)
(1, 15) (8, 15) => (7, 0) (7, 0) (7, 0)
(8, 15) (8, 12) => (0, -3) (0, -3) (0, -3)




# Performance

In [24]:
print('RNG Base Cost')
%timeit ((random.randint(0, 20), random.randint(0, 20)), (random.randint(0, 20), random.randint(0, 20)))
print('\nShift Modulo')
%timeit cardinal_distance_mine((random.randint(0, 20), random.randint(0, 20)), (random.randint(0, 20), random.randint(0, 20)))
print('\nIf Statements')
%timeit cardinal_distance_if((random.randint(0, 20),random.randint(0, 20)), (random.randint(0, 20),random.randint(0, 20)))
print('\nBobs Algorithm')
%timeit cardinal_distance(random.randint(0, 20) + random.randint(0, 20) * 21, random.randint(0, 20) + random.randint(0, 20) * 21)
print('\nBobs Algorithm (cached)')
%timeit cardinal_distance_list[random.randint(0, 20) + random.randint(0, 20) * 21][random.randint(0, 20) + random.randint(0, 20) * 21]

RNG Base Cost
9.52 µs ± 252 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Shift Modulo
11.1 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

If Statements
11.5 µs ± 227 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Bobs Algorithm
15.4 µs ± 567 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Bobs Algorithm (cached)
10.9 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Shift modulo vs cache is pretty close. They are almost withi

# Cache Memory Usage

In [19]:
naturalsize(asizeof.asizeof(cardinal_distance_list), binary=True)

'23.8 MiB'